In [1]:
import pandas as pd
import geopandas as gpd 


In [2]:
parking=pd.read_csv('./parking_tb.csv',encoding='utf-8') 

In [3]:
parking

,parking_nm,p_latitude,p_longitude
0,신선동,35.087052,129.046762
1,봉래시장,35.092927,129.043827
2,유정아파트,35.092663,129.055264
3,청학시장,35.096831,129.059214
4,청학하나,35.096801,129.059279
...,...,...,...
451,녹산산단공영주차장,35.084405,128.840243
452,발렛주차장,35.178674,128.955036
453,차차주차장,35.179756,128.954041
454,동원주차장,35.179229,128.953426


In [4]:
busan_travel=pd.read_csv('./travel_list2.csv',encoding='utf-8') # 관광지정보read
busan_travel

,travel_spot_name,ts_long,ts_lat
0,서면1번가,129.058286,35.153576
1,재난안전체험관,129.105109,35.291822
2,광안리해수욕장,129.118492,35.153791
3,송정해수욕장,129.199640,35.178712
4,해운대해수욕장,129.160308,35.159124
...,...,...,...
559,연제고분판타지축제,129.095056,35.192581
560,다대포어항문화축제,128.972821,35.055623
561,보수동책방골목문화축제,129.027461,35.103125
562,부산고등어축제,129.016761,35.075455


In [5]:
busan_parking_gdf = gpd.GeoDataFrame(parking,geometry=gpd.points_from_xy(parking.p_longitude,parking.p_latitude),crs='epsg:4326')  # WGS84(EPSG:4326) :경도위도좌표계 
busan_travel_gdf = gpd.GeoDataFrame(busan_travel,geometry=gpd.points_from_xy(busan_travel.ts_long,busan_travel.ts_lat),crs='epsg:4326')  

# 부산버스지오df 

In [6]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358)
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182)
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379)
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871)
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912)
...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258)
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562)
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313)
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545)


In [7]:
parking_distances = busan_parking_gdf.to_crs(epsg=5186).geometry.distance(busan_travel_gdf.to_crs(epsg=5186).iloc[0].geometry)

In [8]:
parking_distances

0       7457.901653
1       6859.398162
2       6766.366707
3       6298.780965
4       6302.181484
           ...     
451    21314.206606
452     9813.918461
453     9935.291740
454     9972.163636
455    10155.677040
Length: 456, dtype: float64

In [10]:
near_parking = parking.iloc[parking_distances.idxmin()]['parking_nm']
print(f'{busan_travel_gdf.iloc[0].travel_spot_name}에서 가장가까운 주차장: {near_parking} ({parking_distances.min()}m)')
# 서면1번가에서 가장가까운 버스정류장: 서면1번가 (37.32908144279374m)


서면1번가에서 가장가까운 주차장: 경남공고 뒤 (475.36607344382685m)


In [15]:
# 관광지-가까운 버스정류장 거리계산 함수
def closet_parking(point_geometry):
    distances =  busan_parking_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
    return parking.iloc[distances.idxmin()]['parking_nm'] # idxmin : 최솟값 인덱스찾기

busan_travel_gdf['travel_parking'] = busan_travel_gdf.to_crs(epsg=5186).apply(lambda x: closet_parking(x.geometry), axis=1)

In [12]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry,travel_parking
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358),경남공고 뒤
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182),스포원 주차장(동측)
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379),광안리해수욕장 공영주차장
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871),송정주차장
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912),신일주차장
...,...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258),과정 공영주차장
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562),남항방파제
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313),초량1동업무지
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545),남항방파제


In [13]:
busan_travel_gdf.to_csv('./travel_parking.csv',encoding='utf-8')